# 正弦波入門
- ディジタルな正弦波を作成する
- 正弦波の周波数を指定して「聞くことのできる」波を作る
- waveモジュールを用いたwav出力
- scipyを用いたwav出力
- Audio関数による再生

## セットアップ

In [4]:
import wave
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

In [13]:
OUT_WAVE_FILE = "out_wave.wav" # 出力wavファイル
OUT_SCIPY_WAVE_FILE = "out_wave_scipy.wav" # 出力wavファイル（scipyバージョン）

チャンネル数、量子化ビット数、標本化周波数を指定する

In [6]:
n_channel = 1                   # モノラル
bitdepth = 2                      # 量子化ビット数 16 bit (2 byte)
n_framerate = 16000     # 標本化周波数 (Hz)

## 正弦波の作成

ここで実際の正弦波の仕様を決める必要がある。
まず、連続時間で考えるとして、正弦波信号$x(t)$は
$$
    x(t) = A \sin (2\pi f t)
$$
と書ける。ここで$A$は振幅、$f$は周波数、$t$は連続時間（実数）とする。

標本化周期を$T_s$とすると、標本化周波数$F_s$は$$F_s = \frac{1}{Ts}$$である。

$x(t)$を$t=1/F_s$ごとに標本化する必要があるので（ディジタル化！）、$$t=nT_s = \frac{n}{F_s}\;\; (n = 0, 1, 2, \ldots)$$を$x(t)$の式に「代入」して
$$
    \begin{align}
        x(nT_s) = x[n] = A \sin \left(2\pi f\:n T_s\right) = A \sin \left(2\pi f \frac{n}{F_s}\right) \;\; (n = 0, 1, 2, \ldots) \tag{1}
    \end{align}
$$
を得る。離散時間信号を連続時間信号と区別するために、[ ]を用いた。

続いて連続時間の正弦波の仕様を決める：

In [25]:
freq = 1000                    # 正弦波の周波数 (Hz)
duration = 2                    # 音の継続時間 (sec)
amplitude = 8000          # 正弦波の振幅

Ts = 1.0 / n_framerate           # 標本化周期 (sec)

式(1)に基づいて離散時間の正弦波を作成する。

In [29]:
# 正弦波作成
time = np.arange(0, duration, Ts)  # 継続時間に等しい標本点の作成 → [0 duration]間をTs間隔で区切る
sine_wave = amplitude * np.sin(2 * np.pi * freq * time) # ベクトル入力(time)とスカラー倍(Amplitude)

## 正弦波の保存

In [30]:
# サンプル数
n_frames = len(sine_wave)

In [34]:
# bytesオブジェクトへの変換（今のところは「おまじない」扱いでOK）
sine_wave = sine_wave.astype(np.int16) # floatから符号付き16bit整数への量子化（＝整数への丸め処理）
sound_frames = sine_wave.tobytes() # arrayをバイナリデータ化する = bytesオブジェクトへの変換

In [32]:
# wavの書き込み (waveモジュール)
with wave.open(OUT_WAVE_FILE, "w") as sound:
    sound.setnchannels(n_channel)    # チャネル数
    sound.setsampwidth(bitdepth)     # 量子化ビット数 (byte!)
    sound.setframerate(n_framerate)  # 標本化周波数 (Hz)
    sound.setnframes(n_frames)       # チャネルあたりのサンプル数
    sound.writeframes(sound_frames)  # 音声データの書き込み

In [33]:
# wavの書き込み (scipyモジュール) -> お手軽！
wavfile.write(OUT_SCIPY_WAVE_FILE, n_framerate, sine_wave)

## 正弦波の再生

In [28]:
Audio(sine_wave, rate=n_framerate) # 再生